# HIDDEN MARKOV NEURAL NETWORK: Variational DropConnect example

We test variational DropConnect as regularization criteria

## Import libraries

In [1]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import gzip

import torch
import torch.nn as nn
import torch.nn.functional as F

import pickle

import numpy as np

import matplotlib.pyplot as plt

import random


import BayesianNetworkmu

## Data processing

In [2]:
def train_load_mnist():
    """Load MNIST data from `path`"""
    labels_path = 'train-labels-idx1-ubyte.gz'
    images_path = 'train-images-idx3-ubyte.gz'
        
    with gzip.open(labels_path, 'rb') as lbpath:
        lbpath.read(8)
        buffer = lbpath.read()
        labels = np.frombuffer(buffer, dtype=np.uint8)

    with gzip.open(images_path, 'rb') as imgpath:
        imgpath.read(16)
        buffer = imgpath.read()
        images = np.frombuffer(buffer, 
                               dtype=np.uint8).reshape(
            len(labels), 784).astype(np.float64)
 
    return images, labels

def train_mnist_preprocessing(x, y):
    x = np.float64(x)/126
#     np.save("train_mnist_preprocessed_data", x)
    y = np.int32(y)
#     np.save("train_mnist_preprocessed_target", y)

    return x, y

The files "train-labels-idx1-ubyte.gz" and "train-images-idx3-ubyte.gz" can be downloaded from:

- http://yann.lecun.com/exdb/mnist/

In [3]:
x,y = train_load_mnist()
tr_x, tr_y = train_mnist_preprocessing(x, y)

In [4]:
np.size(tr_y)

60000

## Set the hyper parameters for the training

In [6]:
# set the parameters 
sample_size    = 50000
minibatch_size = 128
epocs          = 10 #600

sliding = 0
T = 1

###########################################################
# Set the network structure

L = 4
architecture = np.array([784, 400, 400, 10])

alpha_k = 0.0
sigma_k = np.exp(-0)
c       = np.exp(6)
p       = 0.25
pi      = 0.5
lr_c = 1e-3
mc_c = 1


## Training procedure 

The prior score is tiny because we are plotting the score at the last epoch, which is scaled during training as suggest in (Blundell et al. 2015). The scaling factor is $\frac{2^{M-i}}{2^{M}-i}$, where $i$ refer to the minibatch we are training on and $M$ refers to the total number of minibatches

In [9]:
########################################################################
# Algorithm training

loss_function = torch.nn.CrossEntropyLoss()

HMMNET = BayesianNetworkmu.torchHHMnet(architecture, alpha_k, sigma_k, c, pi, p, loss_function, sample_size, minibatch_size, epocs, T, sliding, workers = 1)

HMMNET.forward_pass(tr_x[0:50000,:], tr_y[0:50000], tr_x[50000:60000,:], tr_y[50000:60000], lr_c, mc_c)


Time:  1
Prior 2.495046329585567e-114. Loss 2.251445117266791
Performance: 0.5791
Prior 2.5421886020556233e-114. Loss 2.222490797893941
Performance: 0.6314
Prior 2.5759130633257177e-114. Loss 1.9306547559173244
Performance: 0.6708
Prior 2.5990468292538846e-114. Loss 1.4170146330809987
Performance: 0.7427
Prior 2.620160440469432e-114. Loss 0.9946333797344353
Performance: 0.8009
Prior 2.642105871924182e-114. Loss 0.7755934216126118
Performance: 0.8293
Prior 2.652300214723915e-114. Loss 0.7770555670244239
Performance: 0.8524
Prior 2.6674546768800456e-114. Loss 0.42826243999639574
Performance: 0.8699
Prior 2.677322713677541e-114. Loss 0.42928920444239127
Performance: 0.8797
Prior 2.6876552038683196e-114. Loss 0.4022492147227196
Performance: 0.8856


The above simulation took around 15 min on a machine with CPU: 1.80GHz i7-8565U and Ram: 16Gb

## Test procedure

In [6]:
def test_load_mnist():
    """Load MNIST data from `path`"""
    labels_path = 't10k-labels-idx1-ubyte.gz'
    images_path = 't10k-images-idx3-ubyte.gz'
        
    with gzip.open(labels_path, 'rb') as lbpath:
        lbpath.read(8)
        buffer = lbpath.read()
        labels = np.frombuffer(buffer, dtype=np.uint8)

    with gzip.open(images_path, 'rb') as imgpath:
        imgpath.read(16)
        buffer = imgpath.read()
        images = np.frombuffer(buffer, 
                               dtype=np.uint8).reshape(
            len(labels), 784).astype(np.float64)
 
    return images, labels

def test_mnist_preprocessing(x, y):
    x = np.float64(x)/126
    #np.save("test_mnist_preprocessed_data", x)
    y = np.int32(y)
    #np.save("test_mnist_preprocessed_target", y)

    return x, y

In [7]:
x,y = test_load_mnist()
te_x, te_y = test_mnist_preprocessing(x, y)

In [12]:
y_predicted     = np.zeros(len(te_y))
test_performance =0

for t in range(1,2):
    output           = HMMNET.model_list[t].performance( torch.tensor( te_x, dtype = torch.float64 ) )
    output_softmax   = F.softmax(output, dim=1)

    y_predicted = np.array( range(0, 10) )[ np.argmax( output_softmax.data.numpy(), 1 ) ]

    test_performance = sum(te_y == y_predicted)/len(te_y)
    print("Performance: "+ str(test_performance))

Performance: 0.8827
